In [1]:
from PIL import Image, ImageDraw
import numpy as np
from numpy.random import random, randint
from copy import deepcopy
MIN_FIGURES = 2
MAX_FIGURES = 5
MIN_POL_POINTS = 3
MAX_POL_POINTS = 6

In [2]:
class Individual():
    """represents a single instance of a population"""
    
    # the goal image and its height and width
    height = None
    width = None
    goal = None
    
    def __init__(self, figures, x, y):
        self.figures = figures
        self.x = x
        self.y = y
        self.fitness = self.cal_fitness()
        
    def draw(self, show=False):
        """paints all figures on Image and returns it"""
        # setting up canvas
        img = Image.new('RGB', (self.height, self.width))
        d = Image.new('RGBA', (self.height, self.width))
        fig_draw = ImageDraw.Draw(d)
        
        for fig in self.figures:
            # obtaining color and coordinates of a figure
            color = fig.color
            if 'Polygon' in str(fig.__class__):
                coordinates = fig.points
                fig_draw.polygon(coordinates, fill=color, outline=color)
            else:
                coordinates = fig.coordinates
                fig_draw.ellipse(coordinates, fill=color, outline=color)
            
            img.paste(d, mask=d)
            
        if show:
            img.show()
        
        return img
    
    def cal_fitness(self):
        """calculates the fitness (lower is better)"""
        cur = np.array(list(self.draw().getdata())).reshape(self.height, self.width, 3)
        return np.mean(np.abs(self.goal[self.x, self.y] - cur))
    
    def mutate(self):
        """mutation"""
        # checks if it's possible to add a new figure
        if len(self.figures) < MAX_FIGURES:
            p = random()
        
            if p < 0.95:
                # choose some figures and mutate them
                n = randint(1, 4)
                figures = np.random.choice(self.figures, n)
                for fig in figures:
                    fig.mutate()
            else:
                # add a new figure
                self.figures = np.append(self.figures, generate_figures(1)[0])
        else:
            # choose some figures and mutate them
            n = randint(1, 4)
            figures = np.random.choice(self.figures, n)
            for fig in figures:
                fig.mutate()
            
        # recalculate the fitness
        self.fitness = self.cal_fitness()
        
class Polygon():
    """represents a polygon"""
    
    def __init__(self, points, color):
        self.points = points
        self.color = color
        
    def mutate(self):
        if random() <= 0.5:
            # changing the color
            ind = randint(4)
            val = randint(256)
            colors = list(self.color)
            colors[ind] = val
            self.color = tuple(colors)
        else:
            # changine 1 point of the polygon
            ind = randint(len(self.points))
            self.points[ind] = generate_point()
        

        
class Circle():
    """represents the famous russian musician Mike"""
    
    def __init__(self, coordinates, color):
        self.coordinates = coordinates
        self.color = color
    
    def mutate(self):
        """mutating the circle"""
        if random() <= 0.5:
            # changing the color
            ind = randint(4)
            val = randint(256)
            colors = list(self.color)
            colors[ind] = val
            self.color = tuple(colors)
        
        else:
            # changing one point of the box
            ind = randint(4)
            coord = self.coordinates[ind]
            if ind == 1 or ind == 0:
                self.coordinates[ind] = randint(0, self.coordinates[ind + 2])
            else:
                const = Individual.width if ind == 2 else Individual.height
                self.coordinates[ind] = randint(self.coordinates[ind - 2] + 1, const)

def image_processing(img, n_sqr = 64, sqr_size = 8, im_size = 512):
    """ splits the given image into squares"""
    
    t = np.array(list(img.getdata()))
    t = t.reshape(im_size, im_size, 3)

    squares = []
    for i in range(n_sqr):
        for j in range(n_sqr):
            sqr = []
            for cur_y in range(sqr_size):
                for cur_x in range(sqr_size):
                    # creating 1 square
                    sqr.append(t[sqr_size * i + cur_y][sqr_size * j + cur_x])
            
            # appending the square to the list
            squares.append(np.array(sqr).reshape(sqr_size, sqr_size, 3))
    
    # reshaping the list of squares
    t = np.array(squares).reshape(n_sqr, n_sqr, sqr_size, sqr_size, 3)

    return t
    
def generate_point():
    """generates 1 point"""
    x = randint(0, Individual.width)
    y = randint(0, Individual.height)
    
    return (x, y)

def generate_box():
    """generates coordinates of lower left and upper right corners"""
    # lower left corner
    x1 = randint(0, Individual.width - 1)
    y1 = randint(0, Individual.height - 1)
    
    # upper right corner
    x2 = randint(x1 + 1, Individual.width)
    y2 = randint(y1 + 1, Individual.height)
    
    return [x1, y1, x2, y2]

def generate_color():
    """ generates random rgba"""
    r = randint(0, 256)
    g = randint(0, 256)
    b = randint(0, 256)
    a = randint(0, 256)
    return (r, g, b, a)

def generate_circle():
    """generates a circle"""
    coordinates = generate_box()
    color = generate_color()
    
    return Circle(coordinates, color)

def generate_polygon():
    """generates a polygon"""
    n_points = randint(MIN_POL_POINTS, MAX_POL_POINTS + 1)
    
    points = [generate_point() for _ in range(n_points)]
    color = generate_color()
    
    return Polygon(points, color)

def generate_figures(amount = MIN_FIGURES):
    """generates a random array of circles and polygons"""
    figs = []
    
    for _ in range(amount):
        # uncomment the following line to generate ellipses as well
        fig = generate_polygon() #if random() < 0.5 else generate_circle()
        figs.append(fig)
        
    return np.array(figs)

def generate_single_population(x, y, size = 16):
    population = [Individual(generate_figures(MIN_FIGURES), x, y) for _ in range(size)]
    return np.array(population)
    
def generate_populations(sqr_size=8, size=16):
    populations = [[generate_single_population(x, y, size) for x in range(sqr_size)]\
                   for y in range(sqr_size)]
    return np.array(populations)

def roulette_wheel_selection(population):
    # summing all fitnesses
    S = sum(p.fitness for p in population)
    # finding the inverse
    fit_inv = [1 - (p.fitness / S) for p in population]
    # summing inverses
    S = sum(fit_inv)
    # assigning corresponding areas in a wheel
    probs = [f / S for f in fit_inv]
    # roulette wheel selection
    parents = np.random.choice(population, size=2, replace=False, p=probs)
    return parents[0], parents[1]
    
def one_point_crossover(parent1, parent2):
    """1-point crossover"""
    # selecting random point
    point = randint(1, min(len(parent1.figures), len(parent2.figures)))
    # creating 1st child figures
    child1_figures = np.append(parent1.figures[:point], parent2.figures[point:])
    # creating 2nd child figures
    child2_figures = np.append(parent1.figures[point:], parent2.figures[:point])
    x = parent1.x
    y = parent1.y
    # creating children
    child1, child2 = Individual(child1_figures, x, y), Individual(child2_figures, x, y)
        
    return child1, child2
   
def draw(populations, sqr_size, show=False, save=False, generation=None):
    """draws output image"""
    # preparing canvas
    img = Image.new('RGB', (512, 512))
    d = Image.new('RGBA', (512, 512))
    fig_draw = ImageDraw.Draw(d)
    
    for y in range(populations.shape[0]):
        for x in range(populations.shape[1]):
            population = populations[y, x]
            # sorting the population
            population = sorted(population, key = lambda x: x.fitness)
            
            # obtaining individual's image
            square_im = population[0].draw()
            
            # pasting it to output image
            img.paste(square_im, (y * sqr_size + population[0].y, x * sqr_size + population[0].x))
                   
            
    if show:
        img.show()

    if save:
        p = f"{generation}.png"
        img.save(p)
        print ("Saving image")

    return img


def main(path, population_size = 16, sqr_size = 8):
    """
    Currently, works only with images of size 512x512
    """
    # obtaining the image
    img = Image.open(path)
    
    if img.size[0] != 512 or img.size[1] != 512:
        print('Correct the pic size!')
        return 0
    
    n_sqr = 512 // sqr_size
    goal_squares = image_processing(img, n_sqr, sqr_size)
    # filling class vars
    Individual.height = sqr_size
    Individual.width = sqr_size
    Individual.goal = goal_squares
    
    print(f"Generating populations")
    # creating populations
    populations = generate_populations(n_sqr, population_size)
    generation = 0
    print(f"Population has been generated")
    
    #return populations, goal_squares
    ten_perc = int(np.ceil((10 * population_size) / 100))
    

    quard = np.ceil( population_size / 4)
    half = np.ceil(population_size / 2)
    
    while generation <= 1000:
        S_best = 0 # to store fitness sums of best individuals from each population
        S_worst = 0 # to store fitness sums of worst ...
        #print(f"Started generation: {generation}")
        for y in range(n_sqr):
            for x in range(n_sqr):
                # obtaining population of a cell with coordinates (x, y)
                population = populations[y, x]
                # sorting the population
                population = sorted(population, key = lambda x: x.fitness)
                
                # calculating the best and the worst fitness scores
                S_best += population[0].fitness
                S_worst += population[population_size-1].fitness
                
                # the fittest 10% of the old population goes to new population
                new_pop = population[:ten_perc]
                
                # 40% precent of the new is obtained by one point crossover 
                while len(new_pop) < half:
                    p1, p2 = roulette_wheel_selection(population)
                    c1, c2 = one_point_crossover(p1, p2)
                    new_pop = np.append(new_pop, c1)
                    new_pop = np.append(new_pop, c2)
                
                # the rest 50% are mutations of the 1st 50%
                cur = 0
                while len(new_pop) < population_size:
                    copy = deepcopy(new_pop[cur])
                    copy.mutate()
                    new_pop = np.append(new_pop, copy)
                    cur += 1
                    if cur >= half:
                        cur = 0

                populations[y, x] = new_pop
        
        
        if generation % 5 == 0:
            print(f"Current generation: {generation}")
            print(f"best: {S_best}")
            print(f"worst: {S_worst}")
            print("\n")
        
        if generation % 10 == 0:
            draw(populations, sqr_size, save=True, generation=generation)
            
        generation += 1

In [3]:
main('riba.jpg', population_size=8, sqr_size=16)

Generating populations
Population has been generated
Current generation: 0
best: 85843.58723958326
worst: 108202.31640624996


Saving image
Current generation: 5
best: 77702.41927083328
worst: 99740.08333333339


Current generation: 10
best: 72315.64583333318
worst: 94453.5690104167


Saving image
Current generation: 15
best: 68263.59635416666
worst: 92057.67187500012


Current generation: 20
best: 64852.21093750003
worst: 89608.9739583334


Saving image
Current generation: 25
best: 61983.73437499996
worst: 87336.1380208334


Current generation: 30
best: 59678.71484374996
worst: 85681.25520833331


Saving image
Current generation: 35
best: 57619.06770833328
worst: 83404.1575520833


Current generation: 40
best: 55871.382812499956
worst: 83096.35026041667


Saving image
Current generation: 45
best: 54291.792968749956
worst: 81096.73697916673


Current generation: 50
best: 52784.123697916635
worst: 80359.40494791658


Saving image
Current generation: 55
best: 51541.9440104167
worst: 7967

KeyboardInterrupt: 